<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Предобработа-данных" data-toc-modified-id="Предобработа-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработа данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

**План по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка данных

Импортируем библиотеки, которые необходимы для дальнейшей работы:

In [1]:
import pandas as pd 

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Откроем наш датасет. Выведем на экран первые пять объектов, общую информацию о датасете и список характерных значений.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

display(df.head())
print('--------------------------------------------------')
df.info()
print('--------------------------------------------------')
df.describe()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
--------------------------------------------------


,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


В результате первоначального ознакомления с датасетом, мы можем отметить, что датасет состоит из 159571 объектов и двух признаков. Столбец `toxic` будет являться целевым признаком. Т. к. признак категориальный, то будет решаться задача бинарной классификаци классификации. Также по первоначальному изучению можно отметить, что имеется дисбалнс классов (90% - положительные, остальные 10% - отрицательные).

Далее проведем предобработку данных для дальнейшего обучения. 

### Предобработа данных

Проверим наши данные на дубликаты и пропуски

In [3]:
df.duplicated().sum()

0

In [4]:
df.isna().sum()

text     0
toxic    0
dtype: int64

Отлично! Пропусков и дубликатов не обнаружено

Далее проведём лемматизацию и чистку наших комментариев.

In [5]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(lemmatizer.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    return " ".join(cleared_text.split())

df['text'] = df['text'].apply(lemmatize_text)

Разобьём наши данные на признаки и целевой признак. Также разделим нашу выборку на обучающую и тренировочную в соотношении 2 : 3.

In [6]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)

features_train, features_test, target_train, target_test= train_test_split(features,
                                                                               target,
                                                                               test_size=0.4,
                                                                               random_state=12345)

Следующим шагом подготовим наши признаки к обучению: очистим от стоп-слов, векторизируем и приведём в стандартный формат - кодировку Unicode U. 

In [7]:
stopwords = set(stopwords.words('english'))
 
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
 
features_train = count_tf_idf.fit_transform(features_train['text'])
 
features_test = count_tf_idf.transform(features_test['text'])

In [8]:
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((95742, 125610), (63829, 125610), (95742,), (63829,))

## Обучение

Для обучения рассмотрим две модели:
- Решающее дерево (DecisionTreeClassifier)
- Случайный лес (RandomForestClassifier
- Логистическая регрессия (LogisticRegression) 

**DecisionTreeClassifier**

In [9]:
dtc = DecisionTreeClassifier(random_state = 12345)
params = {
   'criterion':['gini', 'entropy'],        
   'max_depth':list(range(1,15,5)) 
}
dtc_gs = GridSearchCV(dtc, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)
target_predictions = dtc_gs.predict(features_train) 

f1 = f1_score(target_train, target_predictions)
print('F1 метрика на тренировочной выборке DecisionTreeClassifier:', f1)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
F1 метрика на тренировочной выборке DecisionTreeClassifier: 0.6129544043595226


**RandomForestClassifier**

In [10]:
rfc = RandomForestClassifier(random_state = 12345, class_weight='balanced')
params = {
   'n_estimators': range(8, 41, 8),
    'max_depth': range(1, 21, 7),
}
rfc_gs = GridSearchCV(rfc, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)
target_predictions = rfc_gs.predict(features_train) 

f1 = f1_score(target_train, target_predictions)
print('F1 метрика на тренировочной выборке RandomForestClassifier:', f1)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
F1 метрика на тренировочной выборке RandomForestClassifier: 0.4158535699585986


**Logistic Regression**

In [11]:
lr = LogisticRegression(random_state = 12345, class_weight='balanced')
params = {
    'penalty': ['l1', 'l2'],
    'C' : [0.5, 1.0, 5.0, 15.0],
    'intercept_scaling' : range(5, 100, 40),
    'solver' : ['liblinear']
 
}
lr_gs = GridSearchCV(lr, params, cv=3, scoring='f1', verbose=True).fit(features_train, target_train)
target_predictions = lr_gs.predict(features_train) 

f1 = f1_score(target_train, target_predictions)
print('F1 метрика на тренировочной выборке LogisticRegression:', f1)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
F1 метрика на тренировочной выборке LogisticRegression: 0.9251306413301663


В итоге мы видим, что лучше всего себя показала модель Логистической регрессии. На тренировочной выборке она показала F1 = 0.92. Чуть хуже себя показала модель Решающего дерева (F1 = 0.61). С обучением хуже всего справилась модель Случайного леса - F1 = 0.42. 

## Выводы

Далее проверим наши модели на тестовой выборке.

In [12]:
predictions_dtc = dtc_gs.predict(features_test)
f1 = f1_score(target_test, predictions_dtc)
print('F1 метрика на тренировочной выборке DecisionTreeClassifier:', f1)

F1 метрика на тренировочной выборке DecisionTreeClassifier: 0.5972478948449373


In [13]:
predictions_rfc = rfc_gs.predict(features_test)
f1 = f1_score(target_test, predictions_rfc)
print('F1 метрика на тренировочной выборке RandomForestClassifier:', f1)

F1 метрика на тренировочной выборке RandomForestClassifier: 0.39168186793141185


In [14]:
predictions_lr = lr_gs.predict(features_test)
f1 = f1_score(target_test, predictions_lr)
print('F1 метрика на тренировочной выборке LogisticRegression:', f1)

F1 метрика на тренировочной выборке LogisticRegression: 0.7647188210279345


**Вывод**

Таким образом, обучив три модели: DecisionTreeClassifier, RandomForestClassifier и DecisionTreeClassofier, мы можем сделать вывод, что лучше всего себя показала модель Логистической регресси. F1 на тестовой выборке равен 0.76. 